# DSC 80: Project 02

### Checkpoint Due Date: Thursday, April 22 11:59:59 PM (Q1-5)

### Final Due Date: Thursday, April 29 11:59:59 PM

---
# Instructions

This Jupyter Notebook contains the statements of the problems and provides code and markdown cells to display your answers to the problems.  
* Like the lab, your coding work will be developed in the accompanying `projectXX.py` file, that will be imported into the current notebook. This code will be autograded.

**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional functions to solve the HW! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `projectXX.py` (much like we do in the notebook).
- Always document your code!

## Checkpoint Instructions

* The checkpoint requires you to turn in **questions 1-5**; 
* **The checkpoint will be graded for *approximate* correctness: easier than the final tests; harder than the doctests.**


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import project02 as proj

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os

# An Investigation into Flight Delays

### The flights dataset

The department of transportation has all flight delays for listed years on their [website](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). There are data for the years 1987 - 2018. See the description of columns in `data/columns.txt`.

This project will look at a single year (2015) to keep the analysis "simple", which is available at the URL below (*NOT* on the data.gov site).


* To download the flights dataset to your computer, use [this link](https://dsc80-fa19-data.s3-us-west-2.amazonaws.com/project02/flight-delays.zip), unzip the file, and place `flights.csv` in your project directory.

* To download the dataset on `datahub.ucsd.edu` (this works on your computer as well!):
    - Open the terminal in datahub ("new > Terminal")
    - Change the directory to where you want your data (e.g. `cd [ASSIGNMENT_PATH]/data`)
    - Download the unzipped dataset using these commands:
        1. `wget https://dsc80-fa19-data.s3-us-west-2.amazonaws.com/project02/flight-delays.zip`
        2. `unzip flight-delays.zip`
    - `flights.csv` should be in the directory.
    
**NOTE: The unzipped files must be in the `project02/data` directory in order for the doctests to work!**

### Creating your datasets

**Question 1**

The flights dataset for 2015 is not small (~600MB). While you could likely load the entire dataset into Pandas on your laptop, if you wanted to work with more than one year, this would quickly become difficult (the data is available for 1987-2018). Therefore, we will filter down the dataset into two smaller files without ever reading the larger dataset fully into memory. We are going to create two smaller datasets:

1. All flights arriving or departing from San Diego International Airport in 2015.
    - You can find a list of all the airport codes in the United States [here](https://www.leonardsguide.com/us-airport-codes.shtml).
2. All flights flown by either JetBlue or Southwest Airline in 2015.

---

To do this, you are going to use the `chunksize=N` keyword in Pandas `read_csv` to read the flights dataset in blocks of `N` lines. When you use this keyword argument, `pd.read_csv(fp, chunksize=N)` becomes a *iterator* that iterates through dataframes of length N until you have reached the end of the dataset. A typical pattern looks like:
```
L = pd.read_csv(filepath, chunksize=1000)
for df in L:
    process(df)
```
Where each `df` is a dataframe of length 1000. 

The processing you are going to do is:
1. Iterate through the dataset, chunk-by-chunk,
2. Filtering out rows of each chunk
3. Incrementally add to a filtered csv file (since the data is perhaps too big to keep in memory). Keep in mind, if you want to keep writing to the same file, the mode='a' keyword in the `.to_csv` method can be helpful when calling it in the loop (a stands for 'append')

---

Write two functions that create the datasets below, using the 'chunking' pattern described above. Your functions should use `chunksize` of 10000.
1. `get_san` which takes in a filepath containing all flights and a filepath where filtered dataset #1 is written. The function should return `None`.
1. `get_jb_sw` which takes in a filepath containing all flights and a filepath where filtered dataset #2 is written. The function should return `None`.

*Remark 1*: **Gradescope autograding servers are quite small and can't load this dataset into memory** -- so your code that reads in the large `flights.csv` dataset *must* work with chunks of the dataset one at a time to pass!

*Remark 2:* You can check your work using the datasets included in the zip file!

Remember to close your file properly!

In [4]:
def get_san(infp, outfp):
    """
    get_san takes in a filepath containing all flights and a filepath where
    filtered dataset #1 is written (that is, all flights arriving or departing
    from San Diego International Airport in 2015).
    The function should return None.

    :Example:
    >>> infp = os.path.join('data', 'flights.test')
    >>> outfp = os.path.join('data', 'santest.tmp')
    >>> get_san(infp, outfp)
    >>> df = pd.read_csv(outfp)
    >>> df.shape
    (53, 31)
    >>> os.remove(outfp)
    """
    dfs = []
    
    chunk = pd.read_csv(infp, chunksize=1000)
    
    for df in chunk:
        chunkdf = df.loc[(df['ORIGIN_AIRPORT'] == 'SAN') | (df['DESTINATION_AIRPORT'] == 'SAN')]
        dfs.append(chunkdf)
        
    concatted = pd.concat(dfs)
    formatted = concatted.reset_index(drop=True)
    formatted.to_csv(outfp,index=False)
    
    return None

In [5]:
infp = os.path.join('data', 'flights.test')
outfp = os.path.join('data', 'santest.tmp')
get_san(infp, outfp)

In [6]:
#df=pd.read_csv(infp)
#df.loc[(df['ORIGIN_AIRPORT'] == 'SAN') | (df['DESTINATION_AIRPORT'] == 'SAN')]

In [7]:
df = pd.read_csv(outfp)
#df

In [8]:
df.shape

(53, 31)

In [9]:
os.remove(outfp)

In [10]:
def get_sw_jb(infp, outfp):
    """
    get_sw_jb takes in a filepath containing all flights and a filepath where
    filtered dataset #2 is written (that is, all flights flown by either
    JetBlue or Southwest Airline in 2015).
    The function should return None.

    :Example:
    >>> infp = os.path.join('data', 'flights.test')
    >>> outfp = os.path.join('data', 'jbswtest.tmp')
    >>> get_sw_jb(infp, outfp)
    >>> df = pd.read_csv(outfp)
    >>> df.shape
    (73, 31)
    >>> os.remove(outfp)
    """
    
    airlines = ['JB','SW']
    dfs = []
    
    chunk = pd.read_csv(infp, chunksize=1000)
    
    for df in chunk:
        chunkdf = df.loc[(df['YEAR'] == 2015)]
        chunkdf = df.loc[(chunkdf['AIRLINE'] == 'B6') | (chunkdf['AIRLINE'] == 'WN')]
        dfs.append(chunkdf)
    
    sw_jb = pd.concat(dfs)
    sw_jb = sw_jb.reset_index(drop = True)
    
    chunkdf.to_csv(outfp,index=False)
    
    return None

In [11]:
infp = os.path.join('data', 'flights.test')
outfp = os.path.join('data', 'jbswtest.tmp')
get_sw_jb(infp, outfp)

In [12]:
df = pd.read_csv(outfp)
#df#.AIRLINE.unique()

In [13]:
df.shape

(73, 31)

In [14]:
os.remove(outfp)

# Flight Delays to/from San Diego

The department of transportation has all flight delays for listed years on their [website](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). 

The zip file at the [URL](https://dsc80-fa19-data.s3-us-west-2.amazonaws.com/project02/flight-delays.zip) contains a file `to_from_san.csv` that consists of all flights either to or from SAN (San Diego) in 2015 -- i.e. the output of Question 1. This dataset should match the dataset that your code returned in question 1.

Read in `to_from_san.csv` using `read_csv` and inspect the dataframe for an initial assessment about the data quality.

In [15]:
# Run this cell
to_from_san_filepath = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(to_from_san_filepath)

In [16]:
flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,978,N693DL,SAN,SLC,615,...,906.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,OO,5608,N930SW,SAN,LAX,615,...,702.0,-5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,WN,823,N7707C,SAN,BWI,620,...,1352.0,-23.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,WN,603,N461WN,SAN,MDW,620,...,1201.0,-29.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,UA,1192,N69804,SAN,DEN,620,...,936.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140409,2015,12,31,4,DL,1792,N1402A,SAN,ATL,2230,...,511.0,-13.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
140410,2015,12,31,4,UA,240,N30401,SAN,ORD,2235,...,405.0,-38.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
140411,2015,12,31,4,DL,1366,N341NW,SAN,DTW,2245,...,530.0,-34.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
140412,2015,12,31,4,UA,498,N37267,SFO,SAN,2252,...,6.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
flights.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [18]:
str(type(flights['ARRIVAL_TIME'].dtypes))

"<class 'numpy.dtype[float64]'>"

In [19]:
flights.DEPARTURE_DELAY[flights.DEPARTURE_DELAY.notna()]

0         -4.0
1         -1.0
2          0.0
3         -3.0
4         -2.0
          ... 
140409    -7.0
140410     5.0
140411   -14.0
140412    -2.0
140413     4.0
Name: DEPARTURE_DELAY, Length: 139010, dtype: float64

### Understanding the data types of the columns

**Question 2**:

* First, classify the *kind* of data each column in `flights` contains. Create a function `data_kinds` of zero variables which outputs a (hard-coded) dictionary of data kinds, keyed by column name, with values `Q`, `O`, `N` (for 'Quantitative', 'Ordinal', or 'Nominal').

* Second, decide the best data *type* for each column. Create a function `data_types` of zero variables which outputs a (hard-coded) dictionary of data types, keyed by column name, with values `str`, `int`, `float`, `bool`. 

*Remark 1*: A column which *should* be `int`s, but contains `NaN`, *must* be a float column. See Lecture 2 notes an explanation of `NaN` and data-types.

*Remark 2*: As with real data, some data processing decisions may be ambiguous here. Make the best decision using the information available to you. It may be helpful to (re)read the relevant [section of the textbook](https://afraenkel.github.io/practical-data-science/03/kinds-of-data.html). 
* Certain answers *may* have more than one correct answer (in these cases, more than one choice gets full credit),
* All answers will be graded for partial credit (some wrong answers are more wrong than other).
There are many columns, so don't worry about the correctness of any given one; do make sure you are thinking about what's contained in a column critically!

In [20]:
def data_kinds():
    """
    data_kinds outputs a (hard-coded) dictionary of data kinds, keyed by column
    name, with values Q, O, N (for 'Quantitative', 'Ordinal', or 'Nominal').

    :Example:
    >>> out = data_kinds()
    >>> isinstance(out, dict)
    True
    >>> set(out.values()) == {'O', 'N', 'Q'}
    True
    """
    #Qtypes = ["<class 'pandas.core.series.Series'>", "<class 'numpy.dtype[float64]'>"]
    #data_dict = {col:'Q' for col in flights.columns if flights.col.dtypes in Qtypes}
    return {'YEAR':'O', 'MONTH':'O', 'DAY':'O', 'DAY_OF_WEEK':'O', 'AIRLINE':'N', 'FLIGHT_NUMBER':'Q',
       'TAIL_NUMBER':'N', 'ORIGIN_AIRPORT':'N', 'DESTINATION_AIRPORT':'N',
       'SCHEDULED_DEPARTURE':'Q', 'DEPARTURE_TIME':'Q', 'DEPARTURE_DELAY':'Q', 'TAXI_OUT':'Q',
       'WHEELS_OFF':'Q', 'SCHEDULED_TIME':'Q', 'ELAPSED_TIME':'Q', 'AIR_TIME':'Q', 'DISTANCE':'Q',
       'WHEELS_ON':'Q', 'TAXI_IN':'Q', 'SCHEDULED_ARRIVAL':'Q', 'ARRIVAL_TIME':'Q',
       'ARRIVAL_DELAY':'Q', 'DIVERTED':'Q', 'CANCELLED':'Q', 'CANCELLATION_REASON':'N',
       'AIR_SYSTEM_DELAY':'Q', 'SECURITY_DELAY':'Q', 'AIRLINE_DELAY':'Q',
       'LATE_AIRCRAFT_DELAY':'Q', 'WEATHER_DELAY':'Q'}

In [21]:
out = data_kinds()
out

{'YEAR': 'O',
 'MONTH': 'O',
 'DAY': 'O',
 'DAY_OF_WEEK': 'O',
 'AIRLINE': 'N',
 'FLIGHT_NUMBER': 'Q',
 'TAIL_NUMBER': 'N',
 'ORIGIN_AIRPORT': 'N',
 'DESTINATION_AIRPORT': 'N',
 'SCHEDULED_DEPARTURE': 'Q',
 'DEPARTURE_TIME': 'Q',
 'DEPARTURE_DELAY': 'Q',
 'TAXI_OUT': 'Q',
 'WHEELS_OFF': 'Q',
 'SCHEDULED_TIME': 'Q',
 'ELAPSED_TIME': 'Q',
 'AIR_TIME': 'Q',
 'DISTANCE': 'Q',
 'WHEELS_ON': 'Q',
 'TAXI_IN': 'Q',
 'SCHEDULED_ARRIVAL': 'Q',
 'ARRIVAL_TIME': 'Q',
 'ARRIVAL_DELAY': 'Q',
 'DIVERTED': 'Q',
 'CANCELLED': 'Q',
 'CANCELLATION_REASON': 'N',
 'AIR_SYSTEM_DELAY': 'Q',
 'SECURITY_DELAY': 'Q',
 'AIRLINE_DELAY': 'Q',
 'LATE_AIRCRAFT_DELAY': 'Q',
 'WEATHER_DELAY': 'Q'}

In [22]:
isinstance(out, dict)

True

In [23]:
set(out.values()) == {'O', 'N', 'Q'}

True

In [24]:
flights.LATE_AIRCRAFT_DELAY[flights.LATE_AIRCRAFT_DELAY.notna()]#.unique()

19         0.0
21         0.0
27        88.0
35         0.0
40         0.0
          ... 
140373    33.0
140374    11.0
140375    34.0
140383     9.0
140395     0.0
Name: LATE_AIRCRAFT_DELAY, Length: 25257, dtype: float64

In [25]:
flights.SCHEDULED_ARRIVAL.isnull().values.any()

False

In [26]:
def data_types():
    """
    data_types outputs a (hard-coded) dictionary of data types, keyed by column
    name, with values str, int, float.

    :Example:
    >>> out = data_types()
    >>> isinstance(out, dict)
    True
    >>> set(out.values()) == {'int', 'str', 'float', 'bool'}
    True
    """

    return {'YEAR':'int', 'MONTH':'int', 'DAY':'int', 'DAY_OF_WEEK':'int', 'AIRLINE':'str', 'FLIGHT_NUMBER':'int',
       'TAIL_NUMBER':'str', 'ORIGIN_AIRPORT':'str', 'DESTINATION_AIRPORT':'str',
       'SCHEDULED_DEPARTURE':'int', 'DEPARTURE_TIME':'float', 'DEPARTURE_DELAY':'float', 'TAXI_OUT':'float',
       'WHEELS_OFF':'float', 'SCHEDULED_TIME':'int', 'ELAPSED_TIME':'float', 'AIR_TIME':'float', 'DISTANCE':'int',
       'WHEELS_ON':'float', 'TAXI_IN':'float', 'SCHEDULED_ARRIVAL':'int', 'ARRIVAL_TIME':'float',
       'ARRIVAL_DELAY':'float', 'DIVERTED':'bool', 'CANCELLED':'bool', 'CANCELLATION_REASON':'str',
       'AIR_SYSTEM_DELAY':'float', 'SECURITY_DELAY':'float', 'AIRLINE_DELAY':'float',
       'LATE_AIRCRAFT_DELAY':'float', 'WEATHER_DELAY':'float'}

In [27]:
out = data_types()
out

{'YEAR': 'int',
 'MONTH': 'int',
 'DAY': 'int',
 'DAY_OF_WEEK': 'int',
 'AIRLINE': 'str',
 'FLIGHT_NUMBER': 'int',
 'TAIL_NUMBER': 'str',
 'ORIGIN_AIRPORT': 'str',
 'DESTINATION_AIRPORT': 'str',
 'SCHEDULED_DEPARTURE': 'int',
 'DEPARTURE_TIME': 'float',
 'DEPARTURE_DELAY': 'float',
 'TAXI_OUT': 'float',
 'WHEELS_OFF': 'float',
 'SCHEDULED_TIME': 'int',
 'ELAPSED_TIME': 'float',
 'AIR_TIME': 'float',
 'DISTANCE': 'int',
 'WHEELS_ON': 'float',
 'TAXI_IN': 'float',
 'SCHEDULED_ARRIVAL': 'int',
 'ARRIVAL_TIME': 'float',
 'ARRIVAL_DELAY': 'float',
 'DIVERTED': 'bool',
 'CANCELLED': 'bool',
 'CANCELLATION_REASON': 'str',
 'AIR_SYSTEM_DELAY': 'float',
 'SECURITY_DELAY': 'float',
 'AIRLINE_DELAY': 'float',
 'LATE_AIRCRAFT_DELAY': 'float',
 'WEATHER_DELAY': 'float'}

In [28]:
isinstance(out, dict)

True

In [29]:
set(out.values()) == {'int', 'str', 'float', 'bool'}

True

### Read in the typed flights data

Read in the flights data using your dictionary of data-types in `read_csv`. This both speeds up parsing, as well as gives you the correct data-types upon reading (which columns would pandas *parse incorrectly* if you didn't use a `dtype` dictionary?)

In [30]:
# Run this cell
dtypes = proj.data_types()
flights = pd.read_csv(to_from_san_filepath, dtype=dtypes)

In [31]:
flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,978,N693DL,SAN,SLC,615,...,906.0,-10.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,OO,5608,N930SW,SAN,LAX,615,...,702.0,-5.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,WN,823,N7707C,SAN,BWI,620,...,1352.0,-23.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,WN,603,N461WN,SAN,MDW,620,...,1201.0,-29.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,UA,1192,N69804,SAN,DEN,620,...,936.0,-9.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140409,2015,12,31,4,DL,1792,N1402A,SAN,ATL,2230,...,511.0,-13.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
140410,2015,12,31,4,UA,240,N30401,SAN,ORD,2235,...,405.0,-38.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
140411,2015,12,31,4,DL,1366,N341NW,SAN,DTW,2245,...,530.0,-34.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
140412,2015,12,31,4,UA,498,N37267,SFO,SAN,2252,...,6.0,-15.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN


**Question 3 (Basic Stats):**

Define a function `basic_stats` that takes a dataframe `flights` and outputs a dataframe that contains statistics for flights arriving/departing for SAN. That is, the output should have two rows, indexed by `ARRIVING` and `DEPARTING`, and have the following columns:

1. The number of arriving/departing flights to/from SAN (`count`).
    - If a flight scheduled to arrive at SAN never arrives, it still counts as an arriving flight.
2. The mean flight (arrival) delay of arriving/departing flights to/from SAN (`mean_delay`).
3. The median flight (arrival) delay of arriving/departing flights to/from SAN (`median_delay`).
4. The airline code of the airline with the single longest flight (arrival) delay among all flights arriving/departing to/from SAN (`airline`).
5. A list of the three months with the greatest number of arriving/departing flights to/from SAN, sorted from greatest to least (`top_months`).

*Remark:* Null values should not be considered when computing statistics; however, think about whether e.g. the average flight delay is likely higher or lower that the "true mean" by making this choice.

*Hint*: Use `groupbby` and the fact that `aggregate` can take in a dictionary as an argument.

In [32]:
fromdf = flights.loc[flights['ORIGIN_AIRPORT'] == 'SAN']

In [33]:
todf = flights.loc[flights['DESTINATION_AIRPORT'] == 'SAN']

In [34]:
fromdf.groupby('MONTH').count().sort_values('DAY',ascending=False).index[:3]

Int64Index([7, 8, 6], dtype='int64', name='MONTH')

In [35]:
def basic_stats(flights):
    """
    basic_stats takes flights and outputs a dataframe that contains statistics
    for flights arriving/departing for SAN.
    That is, the output should have have two rows, indexed by ARRIVING and
    DEPARTING, and have the following columns:

    * number of arriving/departing flights to/from SAN (count).
    * mean flight (arrival) delay of arriving/departing flights to/from SAN
      (mean_delay).
    * median flight (arrival) delay of arriving/departing flights to/from SAN
      (median_delay).
    * the airline code of the airline with the longest flight (arrival) delay
      among all flights arriving/departing to/from SAN (airline).
    * a list of the three months with the greatest number of arriving/departing
      flights to/from SAN, sorted from greatest to least (top_months).

    :Example:
    >>> fp = os.path.join('data', 'to_from_san.csv')
    >>> dtypes = data_types()
    >>> flights = pd.read_csv(fp, dtype=dtypes)
    >>> out = basic_stats(flights)
    >>> out.index.tolist() == ['ARRIVING', 'DEPARTING']
    True
    >>> cols = ['count', 'mean_delay', 'median_delay', 'airline', 'top_months']
    >>> out.columns.tolist() == cols
    True
    """
    from_sd = flights.loc[flights['ORIGIN_AIRPORT'] == 'SAN']
    to_sd = flights.loc[flights['DESTINATION_AIRPORT'] == 'SAN']
    
    q1_dep = from_sd.shape[0]
    q1_arr = to_sd.shape[0]
    
    q2_dep = from_sd['ARRIVAL_DELAY'].mean()
    q2_arr = to_sd['ARRIVAL_DELAY'].mean()
    
    q3_dep = from_sd['ARRIVAL_DELAY'].median()
    q3_arr = to_sd['ARRIVAL_DELAY'].median()
    
    q4_arr = from_sd.loc[from_sd['ARRIVAL_DELAY'] == from_sd['ARRIVAL_DELAY'].max()]['AIRLINE'].values[0]
    q4_dep = to_sd.loc[to_sd['ARRIVAL_DELAY'] == to_sd['ARRIVAL_DELAY'].max()]['AIRLINE'].values[0]
    
    q5_dep = from_sd.groupby('MONTH').count().sort_values('DAY',ascending=False).index.tolist()[:3]
    q5_arr = to_sd.groupby('MONTH').count().sort_values('DAY',ascending=False).index.tolist()[:3]
    
    df = pd.DataFrame({'count':[q1_arr,q1_dep], 'mean_delay':[q2_arr,q2_dep], 'median_delay':[q3_arr,q3_dep], 'airline':[q4_arr,q4_dep], 'top_months':[q5_dep,q5_arr]},index=['ARRIVING','DEPARTING'])
    
    return df

In [36]:
fp = os.path.join('data', 'to_from_san.csv')
dtypes = data_types()
flights = pd.read_csv(fp, dtype=dtypes)
out = basic_stats(flights)
out

,count,mean_delay,median_delay,airline,top_months
ARRIVING,70207,3.676826,-4.0,AA,"[7, 8, 6]"
DEPARTING,70207,3.328988,-5.0,AA,"[7, 8, 6]"


In [37]:
out.index.tolist() == ['ARRIVING', 'DEPARTING']

True

In [38]:
cols = ['count', 'mean_delay', 'median_delay', 'airline', 'top_months']
out.columns.tolist() == cols

True

### Understanding flight delays: Departures, Arrivals, and everything in-between

**Question 4**

Often `DEPARTURE_DELAY` is thought to be the main cause of a flight delay -- i.e., when the flight is late pushing off from the gate. 

However, there are other ways that flights can be late: waiting on the tarmac, headwinds, turbulence, circling a busy airport, and waiting for a gate after landing. First, we will analyze all the ways in which a flight can be delayed.

* First, create a function `depart_arrive_stats` that takes in a dataframe like `flights` and calculates the following quantities in a series, indexed by `late1`, `late2`, `late3`:
    - `late1`: the proportion of flights from/to SAN that leave late, but arrive early or on-time.
    - `late2`: the proportion of flights from/to SAN that leave early, or on-time, but arrive late.
    - `late3`: the proportion of flights from/to SAN that both left late and arrived late.
    
* Second, create a function `depart_arrive_stats_by_month` that takes in a dataframe like `flights` and calculates the quantities above broken down by *month*. That is, the output is a dataframe, indexed by `MONTH`, with columns given by `late1`, `late2`, `late3`.

*Remark 1:* Does this question reveal any data quality issues? Can you pinpoint when these issues occur?

*Remark 2:* A flight is considered late if it departed/arrived any time later than its planned departure/arrival time.

*Remark 3:* You do not need to manually calculate the delay time for those delays that are `NaN`.

In [39]:
flights.LATE_AIRCRAFT_DELAY[flights.LATE_AIRCRAFT_DELAY.notna()]#.unique()

19         0.0
21         0.0
27        88.0
35         0.0
40         0.0
          ... 
140373    33.0
140374    11.0
140375    34.0
140383     9.0
140395     0.0
Name: LATE_AIRCRAFT_DELAY, Length: 25257, dtype: float64

In [40]:
def depart_arrive_stats(flights):
    """
    depart_arrive_stats takes in a dataframe like flights and calculates the
    following quantities in a series (with the index in parentheses):
    - The proportion of flights from/to SAN that
      leave late, but arrive early or on-time (late1).
    - The proportion of flights from/to SAN that
      leaves early, or on-time, but arrives late (late2).
    - The proportion of flights from/to SAN that
      both left late and arrived late (late3).

    :Example:
    >>> fp = os.path.join('data', 'to_from_san.csv')
    >>> dtypes = data_types()
    >>> flights = pd.read_csv(fp, dtype=dtypes)
    >>> out = depart_arrive_stats(flights)
    >>> out.index.tolist() == ['late1', 'late2', 'late3']
    True
    >>> isinstance(out, pd.Series)
    True
    >>> out.max() < 0.30
    True
    """
    
    late1 = flights.loc[(flights['DEPARTURE_DELAY'] > 0) & (flights['ARRIVAL_DELAY'] <= 0)].shape[0] / flights.shape[0]
    
    late2 = flights.loc[(flights['DEPARTURE_DELAY'] <= 0) & (flights['ARRIVAL_DELAY'] > 0)].shape[0] / flights.shape[0]
    
    late3 = flights.loc[(flights['DEPARTURE_DELAY'] > 0) & (flights['ARRIVAL_DELAY'] > 0)].shape[0] / flights.shape[0]
    
    df = pd.Series([late1, late2, late3],index=['late1','late2','late3'])
    return df

In [41]:
fp = os.path.join('data', 'to_from_san.csv')
dtypes = data_types()
flights = pd.read_csv(fp, dtype=dtypes)
out = depart_arrive_stats(flights)
out

late1    0.119853
late2    0.089329
late3    0.278804
dtype: float64

In [42]:
out.index.tolist() == ['late1', 'late2', 'late3']

True

In [43]:
isinstance(out, pd.Series)

True

In [44]:
out.max() < 0.30

True

In [45]:
def depart_arrive_stats_by_month(flights):
    """
    depart_arrive_stats_by_month takes in a dataframe like flights and
    calculates the quantities in depart_arrive_stats, broken down by month

    :Example:
    >>> fp = os.path.join('data', 'to_from_san.csv')
    >>> dtypes = data_types()
    >>> flights = pd.read_csv(fp, dtype=dtypes)
    >>> out = depart_arrive_stats_by_month(flights)
    >>> out.columns.tolist() == ['late1', 'late2', 'late3']
    True
    >>> set(out.index) <= set(range(1, 13))
    True
    """

    return flights.groupby('MONTH').apply(depart_arrive_stats)

In [46]:
fp = os.path.join('data', 'to_from_san.csv')
dtypes = data_types()
flights = pd.read_csv(fp, dtype=dtypes)
out = depart_arrive_stats_by_month(flights)
out

,late1,late2,late3
MONTH,,,
1,0.119063,0.095645,0.270501
2,0.118087,0.096915,0.289925
3,0.130521,0.089240,0.258309
4,0.116475,0.095064,0.264326
5,0.114136,0.083498,0.285151
6,0.121138,0.088248,0.330968
7,0.115371,0.095455,0.335799
8,0.123828,0.086890,0.272450
9,0.110409,0.087890,0.185905


In [47]:
out.columns.tolist() == ['late1', 'late2', 'late3']

True

In [48]:
set(out.index) <= set(range(1, 13))

True

### Flight delays and day of the week

**Question 5**

Next, we'd like to understand the flight traffic to/from SAN by day of the week. Day of the week is specified by integers 1 through 7; verify for yourself which integer corresponds to which day (hint: you have the *date* for each flight as well!).

Next create two functions to understand both the amount of traffic and the average flight delay of flights for each airline by day-of-the week. We both want to understand *presence* each airline has as well as their *performance*.

1. Create a function `cnts_by_airline_dow` that takes in a dataframe like `flights` and outputs a dataframe that answers the following question: Given any `AIRLINE` and `DAY_OF_WEEK`, how many flights were there (in 2015)?


2. Create a function `mean_by_airline_dow` that takes in a dataframe like `flights` and outputs a dataframe that answers the following question: Given any `AIRLINE` and `DAY_OF_WEEK`, what is the average `ARRIVAL_DELAY` (in 2015)?

Both dataframes should have a column for each distinct value of `AIRLINE` and a row for each `DAY_OF_WEEK`.

*Hint:* `pivot_table` should be useful here!

Your output should have the *form* of the table below (not the entries themselves!)

<img src="data/pivot.png" />

In [49]:
#pd.DataFrame(flights.groupby(['DAY_OF_WEEK','AIRLINE'],as_index=False)['AIRLINE'].count())#.pivot(index='DAY_OF_WEEK',columns='AIRLINE')
flights.pivot_table(index='DAY_OF_WEEK',columns='AIRLINE',aggfunc='count')

AIRLINE_DELAY                                           ... YEAR  \
AIRLINE                AA   AS  B6   DL  F9  HA   NK   OO   UA  US  ...   B6   
DAY_OF_WEEK                                                         ...        
1                     347  168  74  180  34  25  124  393  411  77  ...  384   
2                     296  120  87  183  44  23  124  350  471  64  ...  359   
3                     290  130  73  196  45  25  109  327  407  59  ...  359   
4                     355  165  92  222  40  32  116  382  473  85  ...  380   
5                     267  182  80  173  41  30  111  356  400  89  ...  376   
6                     259  177  44  133  30  26  107  314  241  69  ...  270   
7                     298  194  72  161  49  23  124  419  351  85  ...  387   

                                                             
AIRLINE        DL   F9  HA   NK    OO    UA   US   VX    WN  
DAY_OF_WEEK                                                  
1            1753  218  96  531  1859  2369  495  536  9238  
2            1647  210  96  529  1829  2211  463  544  9300  
3            1719  227  96  530  1814  2293  472  542  9328  
4            1729  220  96  531  1835  2328  510  531  9108  
5            1701  208  94  518  1813  2279  505  523  9006  
6            1433  195  94  518  1837  1743  496  365  7201  
7            1726  214  96  532  1889  2172  503  443  8587  

[7 rows x 348 columns]

In [50]:
flights.pivot_table(values='ARRIVAL_DELAY',index='DAY_OF_WEEK',columns='AIRLINE',aggfunc='mean')

AIRLINE,AA,AS,B6,DL,F9,HA,NK,OO,UA,US,VX,WN
DAY_OF_WEEK,,,,,,,,,,,,
1,2.245981,-2.652286,0.928571,-3.096904,3.233945,5.842105,9.222656,7.405257,3.081280,1.764344,12.804878,6.355997
2,0.791037,-4.535474,4.507123,-2.988408,4.975728,7.385417,10.669903,5.109574,5.617431,1.354626,4.645522,5.394194
3,1.811808,-4.030899,2.735043,-2.437098,5.634361,16.833333,6.304264,5.566741,3.686344,-2.145299,0.631970,3.556762
4,4.076552,-2.180995,3.152406,-2.339721,4.531818,7.552083,7.931298,7.971901,4.957124,1.198397,8.277040,6.582300
5,1.235679,-1.040887,3.329759,-5.699528,4.830918,8.021505,7.242718,5.920156,1.756997,3.017928,9.325536,7.961504
6,-0.427912,-3.283673,-1.854478,-3.608726,0.119171,7.159574,6.943026,3.746961,-1.338924,-2.227926,0.101370,1.684625
7,1.611924,-2.318213,2.422043,-4.857392,6.112150,5.229167,8.754753,8.570806,-0.190121,2.257545,12.594533,6.867543


In [51]:
def cnts_by_airline_dow(flights):
    """
    mean_by_airline_dow takes in a dataframe like flights and outputs a
    dataframe that answers the question:
    Given any AIRLINE and DAY_OF_WEEK, how many flights were there (in 2015)?

    :Example:
    >>> fp = os.path.join('data', 'to_from_san.csv')
    >>> flights = pd.read_csv(fp, nrows=100)
    >>> out = cnts_by_airline_dow(flights)
    >>> set(out.columns) == set(flights['AIRLINE'].unique())
    True
    >>> set(out.index) == set(flights['DAY_OF_WEEK'].unique())
    True
    >>> (out >= 0).all().all()
    True
    """
    df = flights.pivot_table(values='DAY',index='DAY_OF_WEEK',columns='AIRLINE',aggfunc='count')
    return df

In [52]:
fp = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(fp, nrows=100)
out = cnts_by_airline_dow(flights)
out

AIRLINE,AA,AS,B6,DL,F9,HA,NK,OO,UA,US,VX,WN
DAY_OF_WEEK,,,,,,,,,,,,
4,8,12,2,11,2,1,4,15,11,4,2,28


In [53]:
#set(flights['AIRLINE'].unique())

In [54]:
#set(out.columns)

In [55]:
set(out.columns) == set(flights['AIRLINE'].unique())

True

In [56]:
set(out.index) == set(flights['DAY_OF_WEEK'].unique())

True

In [57]:
(out >= 0).all().all()

True

In [58]:
def mean_by_airline_dow(flights):
    """
    mean_by_airline_dow takes in a dataframe like flights and outputs a
    dataframe that answers the question:
    Given any AIRLINE and DAY_OF_WEEK, what is the average ARRIVAL_DELAY (in
    2015)?

    :Example:
    >>> fp = os.path.join('data', 'to_from_san.csv')
    >>> flights = pd.read_csv(fp, nrows=100)
    >>> out = mean_by_airline_dow(flights)
    >>> set(out.columns) == set(flights['AIRLINE'].unique())
    True
    >>> set(out.index) == set(flights['DAY_OF_WEEK'].unique())
    True
    """
    df = flights.pivot_table(values='ARRIVAL_DELAY',index='DAY_OF_WEEK',columns='AIRLINE',aggfunc='mean')
    return df

In [59]:
fp = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(fp, nrows=100)
out = mean_by_airline_dow(flights)
out

AIRLINE,AA,AS,B6,DL,F9,HA,NK,OO,UA,US,VX,WN
DAY_OF_WEEK,,,,,,,,,,,,
4,-20.857143,-3.416667,-3.5,-10.636364,-8.0,9.0,-6.75,20.285714,-14.363636,-6.0,-5.0,-1.5


In [60]:
set(out.columns) == set(flights['AIRLINE'].unique())

True

In [61]:
set(out.index) == set(flights['DAY_OF_WEEK'].unique())

True

#flights[flights['CANCELLED'] == 1]### Understanding null values in the flights data

**Question 6 (Missing by Design)**

Now we would like to understand how data is missing in the flights data. First, compute the proportion of each column of `flights` which are non-null. Do not turn this in, but it will be useful information in doing the next few problems.

Recall that a column is *missing by design* if you can determine when the entry of a column is missing based solely on other data in the same row. That is
* there is *no randomness* in determining when an entry is missing.
* you can describe when the column is missing a value with a logical (not random) condition.
* you can express which rows will have missing values in terms of logical statements about the *other* columns in the same row.

For this question, verify the following columns are *missing by design*:
* The column `ARRIVAL_DELAY` is *missing by design*. Create a function `predict_null_arrival_delay` that doesn't depend on the values of `ARRIVAL_DELAY`, that:
    - Takes in a row of the flights data (that is, a Series)
    - Returns `True` if and only if the `ARRIVAL_DELAY` is null; otherwise it returns `False`.
    - Since the function doesn't depend on `ARRIVAL_DELAY`, it should work on a row even if the `ARRIVAL_DELAY` index is dropped.
    - You can check your function by using `flights.drop('ARRIVAL_DELAY', axis=1).apply(predict_null, axis=1)` and compare it to the `ARRIVAL_DELAY` column!


* The column `AIRLINE_DELAY` is *missing by design*. As above, create a function `predict_null_airline_delay` that doesn't depend on the values of `AIRLINE_DELAY`, that:
    - Takes in a row of the flights data (that is, a Series)
    - Returns `True` if and only if the `AIRLINE_DELAY` is null; otherwise it returns `False`.


In [62]:
#flights[flights['CANCELLED'] == 1]

In [63]:
def predict_null_arrival_delay(row):
    """
    predict_null takes in a row of the flights data (that is, a Series) and
    returns True if the ARRIVAL_DELAY is null and otherwise False.

    :param row: a Series that represents a row of `flights`
    :returns: a boolean representing when `ARRIVAL_DELAY` is null.

    :Example:
    >>> fp = os.path.join('data', 'to_from_san.csv')
    >>> flights = pd.read_csv(fp, nrows=100)
    >>> out = flights.drop('ARRIVAL_DELAY', axis=1).apply(predict_null_arrival_delay, axis=1)
    >>> set(out.unique()) - set([True, False]) == set()
    True
    """
    if row['CANCELLED'] == True | row['DIVERTED'] == True:
        return True
    else:
        return False

In [64]:
fp = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(fp, nrows=100)
out = flights.drop('ARRIVAL_DELAY', axis=1).apply(predict_null_arrival_delay, axis=1)
out

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Length: 100, dtype: bool

In [65]:
set(out.unique()) - set([True, False]) == set()

True

In [66]:
flights.drop('ARRIVAL_DELAY',axis=1).apply(predict_null_arrival_delay,axis=1).mean()

0.02

In [67]:
flights['ARRIVAL_DELAY'].isnull().mean()

0.02

In [68]:
#flights[flights['AIRLINE_DELAY'].isna() == False]#.head(60)
#flights

In [69]:
def predict_null_airline_delay(row):
    """
    predict_null takes in a row of the flights data (that is, a Series) and
    returns True if the AIRLINE_DELAY is null and otherwise False. Since the
    function doesn't depend on AIRLINE_DELAY, it should work a row even if that
    index is dropped.

    :param row: a Series that represents a row of `flights`
    :returns: a boolean representing when `AIRLINE_DELAY` is null.

    :Example:
    >>> fp = os.path.join('data', 'to_from_san.csv')
    >>> flights = pd.read_csv(fp, nrows=100)
    >>> out = flights.drop('AIRLINE_DELAY', axis=1).apply(predict_null_airline_delay, axis=1)
    >>> set(out.unique()) - set([True, False]) == set()
    True
    """

    #return row['AIR_SYSTEM_DELAY'] + row['SECURITY_DELAY'] +  row['LATE_AIRCRAFT_DELAY'] + row['WEATHER_DELAY'] != 0
    if row['ARRIVAL_DELAY'] < 15 or row['CANCELLED'] == True or row['DIVERTED'] == True:
        return True
    else:
        return False

In [70]:
fp = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(fp, nrows=100)
out = flights.drop('AIRLINE_DELAY', axis=1).apply(predict_null_airline_delay, axis=1)
out

0      True
1      True
2      True
3      True
4      True
      ...  
95     True
96     True
97     True
98     True
99    False
Length: 100, dtype: bool

In [71]:
set(out.unique()) - set([True, False]) == set()

True

In [72]:
flights.drop('AIRLINE_DELAY',axis=1).apply(predict_null_airline_delay,axis=1).mean()

0.89

In [73]:
flights['AIRLINE_DELAY'].isnull().mean()

0.89

**Question 7 (Missingness Types)**

Now we'd like to determine missingness of the column `DEPARTURE_DELAY`. In particular, we'd like to perform a permutation test to determine the missingness of `DEPARTURE_DELAY` dependent on the column `AIRLINE`.

* Create a function `perm4missing`:
    - that takes in `flights`, a column `col`, and a number `N` and 
    - returns the p-value of the test (using `N` simulations) that determines if `DEPARTURE_DELAY` is MAR dependent on `col`. That is `perm4missing(flights, 'AIRLINE', N)` should return the p-value for the test above.
    - *Remark*: to help your work, create helper functions whose output you can plot, to assess the correctness of your p-value!
    
* Use your function above to determine the columns `col` for which "`DEPARTURE_DELAY` is MAR dependent on `col`" using a significance level of 0.01. Only consider the categorical columns `YEAR`,`DAY_OF_WEEK`, `AIRLINE`,`DIVERTED`, `CANCELLATION_REASON`. Return your answer in a (hard-coded) list returned by a function called `dependent_cols`.

* Create a function `missing_types` of zero variables, which:
    - Returns a Series, indexed by the following columns of `flights`: `CANCELLED`, `CANCELLATION_REASON`, `TAIL_NUMBER`, `ARRIVAL_TIME`.
    - The values should contain the most-likely missingness type of each column. 
    - The values of this Series should be `MD, MCAR, MAR, MNAR, NaN` (use `NaN` if there are no missing values). 


*Hint:* for missingness permutation tests, try using TVD as a test statistic! You are taking the TVD between the distributions of `True` and `False` for those non-numerical columns; `True` being the proportions of null values, `False` being the proportions of non-null values. Check the lecture slides for more information.

In [151]:
def ts_calc(data, col):
    df = data[['DEPARTURE_DELAY',col,'SHUFFLED']]
    #pivotted = pd.DataFrame(df.pivot_table(index='DEPARTURE_DELAY',columns=col,aggfunc='size'))
    pivotted = pd.DataFrame(df.pivot_table(columns=col,aggfunc='size'))
    distr = pivotted / pivotted.sum(axis=0)
    ts = distr.diff(axis=0).iloc[-1].abs().sum() / 2
    return ts

In [166]:
test_flights = flights.copy()
test_flights['SHUFFLED'] = np.random.permutation(flights['DIVERTED'].values)
df = test_flights[['DEPARTURE_DELAY','DIVERTED','SHUFFLED']]
pivotted = pd.DataFrame(df.pivot_table(columns='DIVERTED',aggfunc='size'))
pivotted

,0
DIVERTED,
0,100


In [167]:
distr = pivotted / pivotted.sum(axis=0)
distr

,0
DIVERTED,
0,1.0


In [164]:
ts = distr.diff(axis=0).iloc[-1].abs().sum() / 2
ts

0.0

In [165]:
sim_null(flights, 'DIVERTED')

0.0

In [114]:
#test_flights = flights.copy()
#test_flights['SHUFFLED'] = np.random.permutation(flights['AIRLINE'].values)
#pivotted = pd.DataFrame(test_flights[['DEPARTURE_DELAY','AIRLINE','SHUFFLED']].pivot_table(index='AIRLINE',columns='DEPARTURE_DELAY',aggfunc='size'))
#distr = pivotted / pivotted.sum(axis=0)
#distr
#ts = distr.diff(axis=0).iloc[-1]#.abs().sum() / 2
#ts

In [115]:
#distr = pivotted / pivotted.sum(axis=0)
#distr

In [78]:
#obs = distr.diff(axis=0)#.iloc[:].abs().sum() / 2
#obs

In [123]:
def sim_null(data, col):
    shuffled = np.random.permutation(data[col].values)
    data['SHUFFLED'] = shuffled
    test_stat = ts_calc(data,col)
    return test_stat

In [153]:
def perm4missing(flights, col, N):
    """
    perm4missing takes in flights, a column col, and a number N and returns the
    p-value of the test (using N simulations) that determines if
    DEPARTURE_DELAY is MAR dependent on col.
    :Example:
    >>> fp = os.path.join('data', 'to_from_san.csv')
    >>> flights = pd.read_csv(fp, nrows=100)
    >>> out = perm4missing(flights, 'AIRLINE', 100)
    >>> 0 <= out <= 1
    True
    """
  
    pivotted = pd.DataFrame(flights[['DEPARTURE_DELAY',col]].pivot_table(columns=col,aggfunc='size'))
    distr = pivotted / pivotted.sum(axis=0)
    obs = distr.diff(axis=0).iloc[:,-1].abs().sum() / 2
    differences = []
    
    for trial in range(N):
        trial_null = sim_null(flights,col)
        differences.append(trial_null)

    return np.count_nonzero(differences >= obs) / N

In [125]:
fp = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(fp, nrows=100)
out = perm4missing(flights, 'AIRLINE', 100)
out

0.43000000000000005
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


0.0

In [156]:
perm4missing(flights, 'DIVERTED', 100)

0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


1.0

In [105]:
0 <= out <= 1

True

In [86]:
#to_from_san_filepath = os.path.join('data', 'to_from_san.csv')
#flights = pd.read_csv(to_from_san_filepath)
#flights

In [169]:
perm4missing(flights, 'CANCELLATION_REASON', 100)

0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


1.0

In [170]:
perm4missing(flights, 'YEAR', 100)

0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


1.0

In [171]:
perm4missing(flights, 'DAY_OF_WEEK', 100)

0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


1.0

In [172]:
perm4missing(flights, 'DIVERTED', 100)

0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


1.0

In [173]:
perm4missing(flights, 'AIRLINE', 100)

0.43000000000000005
[0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13]


0.0

In [ ]:
def dependent_cols():
    """
    dependent_cols gives a list of columns on which DEPARTURE_DELAY is MAR
    dependent on.

    :Example:
    >>> out = dependent_cols()
    >>> isinstance(out, list)
    True
    >>> cols = 'YEAR DAY_OF_WEEK AIRLINE DIVERTED CANCELLATION_REASON'.split()
    >>> set(out) <= set(cols)
    True
    """

    return ['AIRLINE','DAY_OF_WEEK']

In [ ]:
out = dependent_cols()
out

In [ ]:
isinstance(out, list)

In [ ]:
cols = 'YEAR DAY_OF_WEEK AIRLINE DIVERTED CANCELLATION_REASON'.split()
set(out) <= set(cols)

In [ ]:
def missing_types():
    """
    missing_types returns a Series
    - indexed by the following columns of flights:
    CANCELLED, CANCELLATION_REASON, TAIL_NUMBER, ARRIVAL_TIME.
    - The values contain the most-likely missingness type of each column.
    - The unique values of this Series should be MD, MCAR, MAR, MNAR, NaN.

    :param:
    :returns: A series with index and values as described above.

    :Example:
    >>> out = missing_types()
    >>> isinstance(out, pd.Series)
    True
    >>> set(out.unique()) - set(['MD', 'MCAR', 'MAR', 'NMAR', np.NaN]) == set()
    True
    """
    indexes = 'CANCELLED CANCELLATION_REASON TAIL_NUMBER ARRIVAL_TIME'.split()
    return pd.Series([np.NaN, 'MD', 'MCAR', 'MD'],index=indexes)

In [ ]:
out = missing_types()
out

In [ ]:
isinstance(out, pd.Series)

In [ ]:
set(out.unique()) - set(['MD', 'MCAR', 'MAR', 'NMAR', np.NaN]) == set()

# Simpson's Paradox: JetBlue vs Southwest

The remainder of the questions investigates the presence of Simpson's paradox in the flights dataset. Read through the final slides of lecture 05, as well as [the book](https://afraenkel.github.io/practical-data-science/05/understanding-aggregations.html#simpsons-paradox) for a summary of Simpson's Paradox and related links.

The csv file `southwest_vs_jetblue.csv` contains all Southwest and JetBlue flights in 2015.

In this dataset, we are going to verify the following occurrences of Simpson's Paradox: For a given set of airports,
* The average departure delay of Southwest is greater than (or less than) the average departure delay of JetBlue.
* Airport by airport, the average departure delay of Southwest is *less* than (or greater than) the average departure delay of JetBlue.

That is, the inequalities of the average flight delays between the two airlines are reversed when viewed at the level of each airport. In fact this reversal holds for *every* airport being considered.

In [ ]:
jb_sw_filepath = os.path.join('data', 'jetblue_or_sw.csv')
dtype = proj.data_types()

# The `usecols` keyword:
# choose *only* the columns you need to reduce your memory footprint!
usecols = ['ORIGIN_AIRPORT','AIRLINE','DEPARTURE_DELAY','CANCELLED']

jb_sw = pd.read_csv(jb_sw_filepath, dtype=dtype, usecols=usecols)

In [ ]:
jb_sw[jb_sw.isnull().any(axis=1)]

In [ ]:
jb_sw[jb_sw['DEPARTURE_DELAY']==0]

**Question 8**

Filter the dataset `jb_sw` to flights *originating* from the following 10 airports: ABQ, BDL, BUR, DCA, MSY, PBI, PHX, RNO, SJC, SLC.

Illustrate Simpson's paradox with this table:
* Calculate the proportion of each airline's flights that are delayed (at each of the 10 airports):
    - Create a function `prop_delayed_by_airline` that takes in a dataframe like `jb_sw` and returns a DataFrame indexed by airline that contains the proportion of each airline's flights that are delayed.
* Calculate these proportions across all airports in the dataset (at each of the 10 airports):
    - Create a function `prop_delayed_by_airline_airport` that takes in a dataframe like `jb_sw` and returns a DataFrame, with columns given by airports, indexed by airline, that contains the proportion of each airline's flights that are delayed at each airport.

*Remark 1:* For the purpose of this question, a canceled flight is **not** considered delayed.

*Remark 2:* Make sure that the functions only work with the columns that are absolutely necessary for the question to avoid out of memory errors!

Verify that Simpson's paradox is present in this output! 

Can you explain *why* Simpson's paradox is occurring? (Hint: where are these airports located? Which have the most flights?)

In [ ]:
airports = ['ABQ', 'BDL', 'BUR', 'DCA', 'MSY', 'PBI', 'PHX', 'RNO', 'SJC', 'SLC']
jb_sw = jb_sw[jb_sw['ORIGIN_AIRPORT'].isin(airports)]
jb_sw

In [ ]:
#jb_sw[jb_sw.isnull().any(axis=1)]

In [ ]:
#non_null = jb_sw.pivot_table(index='AIRLINE',columns='ORIGIN_AIRPORT',aggfunc='count')

In [ ]:
#with_null = jb_sw.pivot_table(index='AIRLINE',columns='ORIGIN_AIRPORT',aggfunc='size')

In [ ]:
#prop_null = 1 - non_null / with_null

In [ ]:
#pd.DataFrame(prop_null.mean(axis=1))

In [ ]:
jb_sw['DELAYED'] = (jb_sw['DEPARTURE_DELAY'] > 0) & (jb_sw['CANCELLED'] == False)
pivotted = jb_sw.pivot_table(index='AIRLINE',columns='ORIGIN_AIRPORT',aggfunc='mean')
pivotted

In [ ]:
def prop_delayed_by_airline(jb_sw):
    """
    prop_delayed_by_airline takes in a dataframe like jb_sw and returns a
    DataFrame indexed by airline that contains the proportion of each airline's
    flights that are delayed.

    :param jb_sw: a dataframe similar to jb_sw
    :returns: a dataframe as above

    :Example:
    >>> fp = os.path.join('data', 'jetblue_or_sw.csv')
    >>> jb_sw = pd.read_csv(fp, nrows=100)
    >>> out = prop_delayed_by_airline(jb_sw)
    >>> isinstance(out, pd.DataFrame)
    True
    >>> (out >= 0).all().all() and (out <= 1).all().all()
    True
    >>> len(out.columns) == 1
    True
    """
    pd.options.mode.chained_assignment = None  # default='warn'
    
    codes = 'ABQ BDL BUR DCA MSY PBI PHX RNO SJC SLC'.split()
    
    jb_sw = jb_sw[['ORIGIN_AIRPORT','AIRLINE','DEPARTURE_DELAY','CANCELLED']]
    jb_sw = jb_sw.query("""ORIGIN_AIRPORT in @codes""")
    
    jb_sw['DELAYED'] = (jb_sw['DEPARTURE_DELAY'] > 0) & (jb_sw['CANCELLED'] == False)
    jb_sw = jb_sw.drop(['DEPARTURE_DELAY','CANCELLED'],axis=1)
    #pivotted = jb_sw.pivot_table(index='AIRLINE',columns='ORIGIN_AIRPORT',values='DELAYED',aggfunc='mean')
    result = jb_sw.groupby("AIRLINE").mean()['DELAYED'].to_frame()
    
    #return pd.DataFrame(pivotted.mean(axis=1))
    #return pivotted
    return result

In [ ]:
fp = os.path.join('data', 'jetblue_or_sw.csv')
jb_sw = pd.read_csv(fp, nrows=100)
out = prop_delayed_by_airline(jb_sw)
out

In [ ]:
isinstance(out, pd.DataFrame)

In [ ]:
(out >= 0).all().all() and (out <= 1).all().all()

In [ ]:
len(out.columns) == 1

In [ ]:
jb_sw = pd.read_csv(fp, nrows=100)
jb_sw = jb_sw[['ORIGIN_AIRPORT','AIRLINE','DEPARTURE_DELAY','CANCELLED']]
jb_sw['DELAYED'] = (jb_sw['DEPARTURE_DELAY'] > 0) & (jb_sw['CANCELLED'] == False)
jb_sw = jb_sw.drop(['DEPARTURE_DELAY','CANCELLED'],axis=1)
jb_sw

In [ ]:
jb_sw[jb_sw['ORIGIN_AIRPORT'] == 'ABQ']

In [ ]:
jb_sw = pd.read_csv(fp, nrows=100)
jb_sw = jb_sw[['ORIGIN_AIRPORT','AIRLINE','DEPARTURE_DELAY','CANCELLED']]
jb_sw['DELAYED'] = (jb_sw['DEPARTURE_DELAY'] > 0) & (jb_sw['CANCELLED'] == False)
jb_sw = jb_sw.drop(['DEPARTURE_DELAY','CANCELLED'],axis=1)

totals = jb_sw.pivot_table(index='AIRLINE',columns='ORIGIN_AIRPORT',aggfunc='size')#.sum(axis=0)
totals

In [ ]:
jb_sw = pd.read_csv(fp, nrows=100)
jb_sw = jb_sw[['ORIGIN_AIRPORT','AIRLINE','DEPARTURE_DELAY','CANCELLED']]
jb_sw['DELAYED'] = (jb_sw['DEPARTURE_DELAY'] > 0) & (jb_sw['CANCELLED'] == False)
jb_sw = jb_sw.drop(['DEPARTURE_DELAY','CANCELLED'],axis=1)

delayed = jb_sw.pivot_table(index='AIRLINE',columns='ORIGIN_AIRPORT',aggfunc='sum')#.sum(axis=1)
delayed

In [ ]:
totals/delayed

In [ ]:
jb_sw = pd.read_csv(fp, nrows=100)
jb_sw = jb_sw[['ORIGIN_AIRPORT','AIRLINE','DEPARTURE_DELAY','CANCELLED']]
jb_sw['DELAYED'] = (jb_sw['DEPARTURE_DELAY'] > 0) & (jb_sw['CANCELLED'] == False)
jb_sw = jb_sw.drop(['DEPARTURE_DELAY','CANCELLED'],axis=1)

mean = jb_sw.pivot_table(index='AIRLINE',columns='ORIGIN_AIRPORT', values='DELAYED',aggfunc='mean')
mean.dropna(axis=1)

In [ ]:
def prop_delayed_by_airline_airport(jb_sw):
    """
    prop_delayed_by_airline_airport that takes in a dataframe like jb_sw and
    returns a DataFrame, with columns given by airports, indexed by airline,
    that contains the proportion of each airline's flights that are delayed at
    each airport.

    :param jb_sw: a dataframe similar to jb_sw
    :returns: a dataframe as above.

    :Example:
    >>> fp = os.path.join('data', 'jetblue_or_sw.csv')
    >>> jb_sw = pd.read_csv(fp, nrows=100)
    >>> out = prop_delayed_by_airline_airport(jb_sw)
    >>> isinstance(out, pd.DataFrame)
    True
    >>> ((out >= 0) | (out <= 1) | (out.isnull())).all().all()
    True
    >>> len(out.columns) == 6
    True
    """
    pd.options.mode.chained_assignment = None  # default='warn'
    
    codes = 'ABQ BDL BUR DCA MSY PBI PHX RNO SJC SLC'.split()
    
    jb_sw = jb_sw[['ORIGIN_AIRPORT','AIRLINE','DEPARTURE_DELAY','CANCELLED']]
    jb_sw = jb_sw.query("""ORIGIN_AIRPORT in @codes""")
    
    jb_sw['DELAYED'] = (jb_sw['DEPARTURE_DELAY'] > 0) & (jb_sw['CANCELLED'] == False)
    jb_sw = jb_sw.drop(['DEPARTURE_DELAY','CANCELLED'],axis=1)

    result = jb_sw.pivot_table(index='AIRLINE',columns='ORIGIN_AIRPORT', values='DELAYED',aggfunc='mean')
    #result = mean#.dropna(axis=1)
    
    return result#.dropna()#[result['DELAY_PROP'] > 0]

In [ ]:
fp = os.path.join('data', 'jetblue_or_sw.csv')
jb_sw = pd.read_csv(fp, nrows=100)
out = prop_delayed_by_airline_airport(jb_sw)
out

In [ ]:
isinstance(out, pd.DataFrame)

In [ ]:
((out >= 0) | (out <= 1) | (out.isnull())).all().all()

In [ ]:
len(out.columns) == 6

In [ ]:
jb_sw_filepath = os.path.join('data', 'jetblue_or_sw.csv')
dtype = proj.data_types()

# The `usecols` keyword:
# choose *only* the columns you need to reduce your memory footprint!
usecols = ['ORIGIN_AIRPORT','AIRLINE','DEPARTURE_DELAY','CANCELLED']

jb_sw = pd.read_csv(jb_sw_filepath, dtype=dtype, usecols=usecols)

In [ ]:
prop_delayed_by_airline(jb_sw)

In [ ]:
prop_delayed_by_airline_airport(jb_sw)

**Question 9**

Your work above illustrates Simpson's paradox on the specific dataset of flights originating from 10 specific airports. However, this still requires you to look at two dataframe to see if the paradox is present. Now, you will create a function that verifies Simpson's paradox in general. You will do this by writing code to compare the two dataframes, instead of inspecting them manually.

Create a function `verify_simpson` that returns a boolean output regarding if the paradox is present.
```
verify_simpson(df, group1, group2, occur)
```
- df is a dataframe (e.g. jb_sw),
- group1 is the first group being aggregated against (e.g. `AIRLINE`),
- group2 is the second group being aggregated against (e.g. `ORIGIN_AIRPORT`),
- occur is a column with values {0, 1}, denoting if an event occurred for that individual.
  (e.g. "1 if flight was delayed" and "0 if flight was not delayed")

`verify_simpson` should return `True` only if there is a reversal for *every* value of `group2` (e.g. for every airport).

Example:

Consider the following dataframe `df` with columns `treatment`, `stone_size`, and `success`:

|treatment|stone_size|success|
|---|---|---|
|A|small|1|
|B|small|1|
|...|...|...|
|A|large|0|
|B|small|0|
|B|small|1|

`df` corresponds to the following diagram:
<img src="https://miro.medium.com/max/996/1*IfVjfdGD7RPwLDC6WzT9Uw.png" style="width: 300px"/>

Here, `verify_simpson(df, 'treatment', 'stone_size', 'success')` should return `True`.

Verify that you function works on the previous question!

In [ ]:
def verify_simpson(df, group1, group2, occur):
    """
    verify_simpson verifies whether a dataset displays Simpson's Paradox.

    :param df: a dataframe
    :param group1: the first group being aggregated
    :param group2: the second group being aggregated
    :param occur: a column of df with values {0,1}, denoting
    if an event occurred.
    :returns: a boolean. True if simpson's paradox is present,
    otherwise False.

    :Example:
    >>> df = pd.DataFrame([[4,2,1], [1,2,0], [1,4,0], [4,4,1]], columns=[1,2,3])
    >>> verify_simpson(df, 1, 2, 3) in [True, False]
    True
    >>> verify_simpson(df, 1, 2, 3)
    False
    """

    group1_values = df.groupby(group1).mean()[occur].diff().iloc[-1] < 0
    
    group2 = df.pivot_table(index=group1, columns=group2, aggfunc='mean',values=occur)
    group2_values = group2.diff().iloc[-1] < 0
    
    if all(group2_values):
        return group1_values != all(group2_values)
    elif all(group2_values == False):
        return group1_values != all(group2_values)
    else:
        return False

In [ ]:
df = pd.DataFrame([[4,2,1], [1,2,0], [1,4,0], [4,4,1]], columns=[1,2,3])

In [ ]:
verify_simpson(df, 1, 2, 3) in [True, False]

In [ ]:
verify_simpson(df, 1, 2, 3)

### Bonus problem (worth zero points)

This question is for fun and explores a very data-science type problem: can we automate finding examples of Simpson's Paradox? This is an active area of research (see for example: https://arxiv.org/pdf/1801.04385.pdf), but is a very accessible problem. While totally optional, this question can lead to pretty interesting self-driven projects!

**Question 10 (Searching for Simpson's Paradox):**

As you observed from the reading in the lecture notes, Simpson's Paradox often occurs due to some confounding factor among the columns of a dataset. In the case of gender balance in student admissions to academic departments at UC Berkeley, the confounding factor was the admission rate (i.e. how hard it is to gain admission to a department).

What might be a confounding factor be for flight delays among airports in question 8? Now you are going to write code to discover instances of Simpson's Paradox; that is, you will programmatically find interesting relationships present in the data.

Given the dataset `jb_sw`, we'd like to find new groups of airports, as in question 8, for which the statistics of flight delays between JetBlue and Southwest satisfy Simpson's Paradox.

Create a function `search_simpsons` that takes in the `jb_sw` dataset and a number `N`, and returns a list of `N` airports for which the proportion of flight delays between JetBlue and Southwest satisfies Simpson's Paradox.
- Only consider airports that have '3 letter codes',
- Only consider airports that have at least one JetBlue *and* Southwest flight.

*Remark 1:* Iterate through groups of airports of size `N` using `itertools.combinations` until you find a group that works. Make sure your function finishes, even if it doesn't find something.

*Remark 2:* You should be using your work from Question 9!

In [ ]:
def search_simpsons(jb_sw, N):
    """
    search_simpsons takes in the jb_sw dataset and a number N, and returns a
    list of N airports for which the proportion of flight delays between
    JetBlue and Southwest satisfies Simpson's Paradox.

    Only consider airports that have '3 letter codes',
    Only consider airports that have at least one JetBlue and Southwest flight.

    :Example:
    >>> fp = os.path.join('data', 'jetblue_or_sw.csv')
    >>> jb_sw = pd.read_csv(fp, nrows=1000)
    >>> pair = search_simpsons(jb_sw, 2)
    >>> len(pair) == 2
    True
    """

    return ...

In [ ]:
fp = os.path.join('data', 'jetblue_or_sw.csv')
jb_sw = pd.read_csv(fp, nrows=1000)
pair = search_simpsons(jb_sw, 2)

In [ ]:
len(pair) == 2

# Congratulations, you finished the project!

### Before you submit:
* Be sure you run the doctests on all your code in project02.py

### To submit:
* **Upload the .py file to gradescope**